# Solving the N-Puzzle problem

The objective of this exercise is the application of search methods, with emphasis on informed search methods and the A* algorithm, to solve the well-known N-Puzzle problem. The desired objective state for the puzzle is as follows (0 represents the empty space):

| 9 |   | Puzzle |
|---|---|--------|
| 1 | 2 | 3      |
| 4 | 5 | 6      |
| 7 | 8 | 0      |


| 16 |    |    | Puzzle |
|----|----|----|--------|
| 1  | 2  | 3  | 4      |
| 5  | 7  | 7  | 8      |
| 9  | 10 | 11 | 12     |
| 13 | 14 | 15 | 0      |

a) Formulate the problem as a search problem indicating the state representation, operators (their names, preconditions, effects, and cost), initial state, and objective test. 

**State** - a matrix, $M$, $N \times N$ to represent the puzzle and a pair $(row, column)$ that indicates where the empty slot is. For each value, $V$, in the matrix, $V \in \{1..N^2-1\}$. $row, column \in \{0..N-1\}$

**Initial state** - any valid valid matrix and the position of the empty slot.

**Operators**

| Name  | Pre conditions   | Effects                                                       | Cost |
|-------|------------------|---------------------------------------------------------------|------|
| up    | $row > 0$        | $row = row - 1 \land M[row, column]=M[row - 1, column] \land M[row - 1, column] = 0$ | 1    |
| down  | $row < N - 1$    | $row = row + 1 \land M[row, column]=M[row + 1, column] \land M[row + 1, column] = 0$ | 1    |
| left  | $column > 0$     | $column = column - 1 \land M[row, column]=M[row, column - 1] \land M[row, column - 1] = 0$ | 1    |
| right | $column < N - 1$ | $column = column + 1 \land M[row, column]=M[row, column + 1] \land M[row, column + 1] = 0$ | 1    |

**Objective state** - the flattened matrix is a ordered list from $1$ to $N^2-1$ except for the empty slot that is represented by a $0$ that is the last element.

In [1]:
%reload_ext autoreload
%autoreload 2

from search import *
from performance import *
import sys
import numpy as np

sys.setrecursionlimit(10**6)

In [2]:
# Utilities

def swap_positions(matrix, position, offset_row = 0, offset_column = 0):
    (row, column) = position
    
    new_matrix = [row.copy() for row in matrix]
    
    next_row = row + offset_row
    next_column = column + offset_column

    new_matrix[row][column] = new_matrix[next_row][next_column]
    new_matrix[next_row][next_column] = 0

    return (new_matrix, (next_row, next_column))

In [3]:
# Operators

def up(matrix, position):
    if position[0] == 0:
        return False

    return swap_positions(matrix, position, offset_row=-1)


def down(matrix, position):
    if position[0] == len(matrix) - 1:
        return False

    return swap_positions(matrix, position, offset_row=1)


def left(matrix, position):
    if position[1] == 0:
        return False

    return swap_positions(matrix, position, offset_column=-1)


def right(matrix, position):
    if position[1] == len(matrix) - 1:
        return False

    return swap_positions(matrix, position, offset_column=1)


# Objective state

def objective_test(matrix, _):
    flattened = np.array(matrix).flatten().tolist()
    
    if flattened.pop(-1) != 0:
        return False
    
    return sorted(flattened) == flattened

functions = [up, down, left, right]

In [4]:
# Pretty print solution

def prettyprint_solution(path, cost):
    if path == None:
        print("No solution found")
        return
    
    print(f"Solution found with cost {cost}")
    
    directions = list(map(lambda node: node.function if node.function != None else "Start", path))
    
    print("Directions:", ", ".join(directions))

In [5]:
initial_state_1 = ([
        [1,2,3],
        [5,0,6],
        [4,7,8]
    ], (1,1))

initial_state_2 = ([
        [1,3,6],
        [5,2,0],
        [4,7,8]
    ], (1,2))

initial_state_3 = ([
        [1,6,2],
        [5,7,3],
        [0,4,8]
    ], (2,0))

initial_state_4 = ([
        [5,1,3,4],
        [2,0,7,8],
        [10,6,11,12],
        [9,13,14,15]
    ], (1,1))

**Breadth first search approach** (in this case identical to Uniform Cost) removing states that are repeated within a maximum number of parent nodes of 6

In [6]:
print("Problem 1 3x3")
measure_search(bfs, prettyprint_solution, initial_state_1, functions, objective_test)

print("Problem 2 3x3")
measure_search(bfs, prettyprint_solution, initial_state_2, functions, objective_test)

print("Problem 3 3x3")
measure_search(bfs, prettyprint_solution, initial_state_3, functions, objective_test)

Problem 1 3x3
Solution found with cost 4
Directions: Start, left, down, right, right

bfs took 0.0009992122650146484 seconds to run

Problem 2 3x3
Solution found with cost 7
Directions: Start, up, left, down, left, down, right, right

bfs took 0.003000974655151367 seconds to run

Problem 3 3x3
Solution found with cost 10
Directions: Start, right, up, up, right, down, left, left, down, right, right

bfs took 0.015999794006347656 seconds to run



In [7]:
# Defining heuristics

def heuristic_1(matrix, _):
    length = len(matrix)
    heuristic_value = 0

    for row in range(0, length):
        for column in range(0, length):
            value = matrix[row][column]
            expected = row * length + column + 1
            
            if value == 0 or value == expected:
                continue
           
            heuristic_value += 1

    return heuristic_value


def heuristic_2(matrix, _):
    length = len(matrix)
    heuristic_value = 0

    for row in range(0, length):
        for column in range(0, length):
            value = matrix[row][column]
            expected = row * length + column + 1
            
            if value == 0 or value == expected:
                continue
           
            value_row = (value - 1) // length
            value_column = (value - 1) % length

            heuristic_value += abs(value_row - row) + abs(value_column - column)

    return heuristic_value


In [8]:
print("Problem 1 3x3 H1")
measure_search(a_star, prettyprint_solution, initial_state_1, functions, objective_test, heuristic_1)

print("Problem 2 3x3 H1")
measure_search(a_star, prettyprint_solution, initial_state_2, functions, objective_test, heuristic_1)

print("Problem 3 3x3 H1")
measure_search(a_star, prettyprint_solution, initial_state_3, functions, objective_test, heuristic_1)

print("Problem 4 4x4 H1")
measure_search(a_star, prettyprint_solution, initial_state_4, functions, objective_test, heuristic_1)

print("Problem 1 3x3 H2")
measure_search(a_star, prettyprint_solution, initial_state_1, functions, objective_test, heuristic_2)

print("Problem 2 3x3 H2")
measure_search(a_star, prettyprint_solution, initial_state_2, functions, objective_test, heuristic_2)

print("Problem 3 3x3 H2")
measure_search(a_star, prettyprint_solution, initial_state_3, functions, objective_test, heuristic_2)

print("Problem 4 4x4 H2")
measure_search(a_star, prettyprint_solution, initial_state_4, functions, objective_test, heuristic_2)

Problem 1 3x3 H1
Solution found with cost 4
Directions: Start, left, down, right, right

a_star took 0.0 seconds to run

Problem 2 3x3 H1
Solution found with cost 7
Directions: Start, up, left, down, left, down, right, right

a_star took 0.0 seconds to run

Problem 3 3x3 H1
Solution found with cost 10
Directions: Start, right, up, up, right, down, left, left, down, right, right

a_star took 0.0029997825622558594 seconds to run

Problem 4 4x4 H1
Solution found with cost 10
Directions: Start, left, up, right, down, down, left, down, right, right, right

a_star took 0.0009996891021728516 seconds to run

Problem 1 3x3 H2
Solution found with cost 4
Directions: Start, left, down, right, right

a_star took 0.0 seconds to run

Problem 2 3x3 H2
Solution found with cost 7
Directions: Start, up, left, down, left, down, right, right

a_star took 0.0009999275207519531 seconds to run

Problem 3 3x3 H2
Solution found with cost 10
Directions: Start, right, up, up, right, down, left, left, down, right,